# 残差网络 Residual Networks

感谢大家来我的网站学习https://www.captainbed.net


理论上来说，越深的神经网络就能解决越复杂的问题；但是，在实践中，越深的神经网络就越难被训练好。而残差网络可以帮助我们训练深神经网络。

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Colab Notebooks/explore/ai/18_residual_networks")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from tensorflow.keras.utils import plot_model
from resnets_utils import *

import scipy.misc

from matplotlib.pyplot import imshow
%matplotlib inline


## 1 - 是什么阻碍了深度神经网络

随着先人们年复一年的努力，我们构建的神经网络也越来越深了，从最初的几层，到现在的百多层。为什么要构建越来越深的神经网络呢？因为越深的神经网络，就能解决越复杂的问题，就能学习到更多不同层次的特征，前面的层会学到基本的特征，例如线条，后面的层可以学到复杂的特征，例如人脸。

但是，神经网络越深，梯度消失就越严重（偶尔也会是梯度爆炸问题），导致神经网络学得越来越慢，越来越迟钝。就像一个弱智大块头，虽然身体很大，但是智商很低。虚胖~~

所以，我们要使用残差网络来解决这个问题。

## 2 - 构建残差网络

残差网络是由残差块组成的。下面左图是传统的网络块，右边的是残差网络块。其实就是在传统的网络块上添加一条小路，以便让激活值和梯度值可以跳层传播，以此来避免梯度消失和爆炸（如果不明白这句话，那么回头多看几遍梯度消失的文章）。:  

<img src="images/skip_connection_kiank.png" style="width:650px;height:200px;">
<caption><center> <u> <font color='purple'> **图 2** </u><font color='purple'>  : 残差块 <br> </center></caption>

在有些文献中，说残差块能更好的独立的学到某个特征，以此避免了梯度消失。这里只是顺便提一下，大家可以不深究。

在实现残差块时，会有两种情况，一种是要跳跃传递的矩阵与目标层矩阵的维度一致时，另一种情况是跳传矩阵与目标层矩阵不一致时，不一致时就需要变换矩阵的维度。我们先看维度一致时的情况。

### 2.1 - 维度一致时的残差块

下图小路的左边的网络层的激活值与小路右边的网络层的激活值的维度是一致的，所以可以直接跳过去。就是说$a^{[l]}$)与$a^{[l+2]}$的维度是一致的。从下图中可以看出有两条路，一条直线主路，一条弧线小路:

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **图 3** </u><font color='purple'>  : . </center></caption>

上图是跳了2层，其实还可以跳更多层，下图就跳了3层: 

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **图 4** </u><font color='purple'>  :</center></caption>

In [ ]:
# 维度相同时的残差块
def identity_block(X, f, filters, stage, block):
    """
    实现了图4中展示的残差块

    参数:
    X -- 要跳跃的激活值矩阵
    f -- 整型。指示卷积层的窗口大小
    filters -- 整型数组，指示残差块中的卷积层的过滤器的个数
    stage -- 整型。用来辅助给网络层取名。
    block -- 字符串。用来辅助给网络层取名。

    返回:
    X -- 残差块的最终输出矩阵
    """

    # 取一些名字而已，无关紧要
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # 获取每一个卷积层对应的过滤器的个数
    F1, F2, F3 = filters
    
    # 定义参数初始化的方法
    initializer = tf.keras.initializers.GlorotUniform

    # 保存输入的激活值，以便后面跳插入到后面的网络层中
    X_shortcut = X

    # 这里我们使用functional API方式来构建模型
    # 因为在开头处已经从keras里面导入了Conv2D，所以这里我们就可以直接使用了，相当于tensorflow.keras.layers.Conv2D
    # 主路中的第一组网络层，就是图4的第一组绿橙黄小方块
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a',
               kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)


    # 主路中的第二组网络层，就是图4的第二组绿橙黄小方块
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', 
               kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # 主路中的第三组网络层，图4的第三组绿橙小方块
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c',
               kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)
    # 这一步就是实现小路的地方。其实就是简单的将前面层的激活值X_shortcut与第三组网络层的输出激活值合并在一起
    # 然后将合并的激活值向下传入到激活函数中，进入到后面的神经网络中去
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
    np.random.seed(1)
    X = tf.cast(np.random.randn(3, 4, 4, 6),tf.float32)
    A = identity_block(X, f=2, filters=[2, 4, 6], stage=1, block='a')
    tf.print("out = ", A[1][1][0])


out =  [0 0 1.34546745 2.03181791 0 1.32467544]


## 2.2 - 维度不同时的情况

当维度不同时，我们就不能直接将前面的激活值和后面的激活值矩阵合并在一起，所以需要在小路上加个卷积层来改变前面的激活矩阵的维度。如下图所示,小路上加了一个conv2d卷积层: 

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">
<caption><center> <u> <font color='purple'> **图 5** </u><font color='purple'>  :</center></caption>



In [ ]:
# 实现图5中的残差块

def convolutional_block(X, f, filters, stage, block, s=2):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    F1, F2, F3 = filters
    
    initializer = tf.keras.initializers.GlorotUniform
    
    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', 
               kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', 
               kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', 
               kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # 在小路上面加上一个卷积层和一个BatchNormalization
    # 卷积层会改变X_shortcut的维度，这样一来，就能与X矩阵合并在一起了
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', 
                        kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # 将变维后的X_shortcut与X合并在一起
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X


In [ ]:
    np.random.seed(1)
    X = tf.cast(np.random.randn(3, 4, 4, 6),dtype=tf.float32)
    A = convolutional_block(X, f=2, filters=[2, 4, 6], stage=1, block='a')
    tf.print("out = ", (A[1][1][0]))


out =  [0 0 0 0.892965555 0 0.198827311]


## 3 - 构建ResNet50(一个50层的残差网络)

接下来带领大家实现一个50层的残差网络。如下图所示。里面的ID BLOCK是指维度相同时的残差块，ID BLOCK x3表示有3组这样的残差块。CONV BLOCK是指维度不同时的残差块。我们将中间的残差块群范围了5个阶段，stage1...stage5。.

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">
<caption><center> <u> <font color='purple'> **图 6** </u><font color='purple'>  : **ResNet-50模型** </center></caption>


In [ ]:
# 实现ResNet50

def ResNet50(input_shape=(64, 64, 3), classes=6):
    """
    参数:
    input_shape -- 输入的图像矩阵的维度
    classes -- 类别数量

    Returns:
    model -- 网络模型
    """

    # 根据输入维度定义一个输入向量
    X_input = Input(input_shape)
    
    initializer = tf.keras.initializers.GlorotUniform

    # 用0填充输入向量的周边
    X = ZeroPadding2D((3, 3))(X_input)

    # 实现第一个阶段。Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    # Stage 3 
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # 平均池化层
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

    # 扁平化激活值矩阵，对接上全连接层，softmax层
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), 
              kernel_initializer=initializer(seed=0))(X)

    # 构建模型
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape=(64, 64, 3), classes=6)

编译模型

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

编译好后，就可以开始训练这个模型了。下面我们先把数据集加载进来。


<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'> **图7** </u><font color='purple'>  : **手势数据集** </center></caption>


In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

X_train = X_train_orig / 255.
X_test = X_test_orig / 255.

Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

number of training examples = 1080
number of test examples = 120
X_train shape: (1080, 64, 64, 3)
Y_train shape: (1080, 6)
X_test shape: (120, 64, 64, 3)
Y_test shape: (120, 6)


由于模型太大，训练时间太长，我们下面只训练2个epochs。仅仅是2个epochs，都要花十多分钟，去看个小视频再回来看结果吧！！！

In [ ]:
# 之前我们在使用fit时，是传入了dataset对象，在这里我们分别传入了训练样本和训练标签。这两种方法都是可用的。
# 具体fit的用法可以查看官方文档https://tensorflow.google.cn/api_docs/python/tf/keras/Model#fit
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

Epoch 1/2
34/34 [==============================] - 257s 7s/step - loss: 2.7048 - accuracy: 0.3176
Epoch 2/2
34/34 [==============================] - 243s 7s/step - loss: 0.5782 - accuracy: 0.8139


epoch2的loss为0.5，比epoch1的要小，epoch2的精准度维0.81，比epoch1的要大。说明训练的越多，网络精准度越高了。

下面用测试集看看模型的精准度

In [ ]:
preds = model.evaluate(X_test, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 5s 798ms/step - loss: 4.6475 - accuracy: 0.1667
Loss = 4.6474504470825195
Test Accuracy = 0.1666666716337204


精准度很低。因为我们只训练了2个epochs。当然，你可以修改上面的代码，训练多个epochs，那么精准度会上去的。我试过，在CPU上训练20个epochs后，精准度会有明显提升，不过20个epochs要花1个小时左右的时间。

为了给大家展示结果，我在GPU上将ResNet50模型训练好了。使用下面的代码加载这个训练好的模型，然后在测试集上面展示结果。

In [ ]:
model = load_model('ResNet50.h5') 

In [ ]:
preds = model.evaluate(X_test, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

4/4 [==============================] - 5s 796ms/step - loss: 0.5302 - accuracy: 0.8667
Loss = 0.5301783680915833
Test Accuracy = 0.8666666746139526


可以看到，精准度达到了0.8，还不错。